## Q1

In [ ]:
######### Solution #########

# 我不懂为什么Seb不对齐 杀杀杀

#question 1
from enum import Enum
import time
import random
from contextlib import redirect_stdout
import io
import string
import datetime

class InvalidValueException(Exception):
    pass

class SecurityDefinitionType(Enum):
    FUTURES = 0,
    OPTIONS = 1,
    FX = 2

def negative_value_checker(func):
    def wrapper(*args):
        if len(args) == 6:
            if args[5] < 0:
                raise InvalidValueException("Invalid values not accepted.")
        elif len(args) == 5:
            if args[4] < 0:
                raise InvalidValueException("Invalid values not accepted.")
        elif len(args) == 4:
            if args[3] < 0:
                raise InvalidValueException("Invalid values not accepted.")
        return func(*args)
    return wrapper

class FeedFuturesSecurityDefinition:
    @negative_value_checker
    def __init__(self, symbol_name, id, tick_value, tick_size):
        self.__symbol_name = symbol_name
        self.__id = id
        self.__tick_value = tick_value
        self.__tick_size = tick_size

    def getSymbolName(self):
        return self.__symbol_name

    def getId(self):
        return self.__id

    def getTickValue(self):
        return self.__tick_value

    def getTickSize(self):
        return self.__tick_size

class FeedOptionsSecurityDefinition:
    @negative_value_checker
    def __init__(self, symbol_name, id, expiry, tick_value, tick_size):
        self.__symbol_name = symbol_name
        self.__id = id
        self.__expiry = expiry
        self.__tick_value = tick_value
        self.__tick_size = tick_size

    def getSymbolName(self):
        return self.__symbol_name

    def getId(self):
        return self.__id

    def getExpiry(self):
        return self.__expiry

    def getTickValue(self):
        return self.__tick_value

    def getTickSize(self):
        return self.__tick_size

class FeedFxSecurityDefinition:
    @negative_value_checker
    def __init__(self, symbol_name, id, pip_scale):
        self.__symbol_name = symbol_name
        self.__id = id
        self.__pip_scale = pip_scale

    def getSymbolName(self):
        return self.__symbol_name

    def getId(self):
        return self.__id

    def getPipScale(self):
        return self.__pip_scale

class SecurityDefinition:
    def __init__(self, security_definition_type):
        self.__security_definition_type = security_definition_type
        self.__symbol_name = ""
        self.__id = 0

    @property
    def security_definition_type(self):
        return self.__security_definition_type

    @security_definition_type.setter
    def security_definition_type(self, security_definition_type):
        self.__security_definition_type = security_definition_type

    @property
    def symbol_name(self):
        return self.__symbol_name

    @symbol_name.setter
    def symbol_name(self, symbol_name):
        self.__symbol_name = symbol_name

    @property
    def id(self):
        return self.__id

    @id.setter
    def id(self, id):
        self.__id = id

class FuturesSecurityDefinition(SecurityDefinition):
    def __init__(self):
        super().__init__(SecurityDefinitionType.FUTURES)
        self.__tick_value = 0
        self.__tick_size = 0

    @property
    def tick_value(self):
        return self.__tick_value

    @tick_value.setter
    def tick_value(self, tick_value):
        self.__tick_value = tick_value

    @property
    def tick_size(self):
        return self.__tick_size

    @tick_size.setter
    def tick_size(self, tick_size):
        self.__tick_size = tick_size

    def __str__(self):
        return "DoNotImplement"

    def __repr__(self):
        return "DoNotImplement"

class OptionsSecurityDefinition(SecurityDefinition):
    def __init__(self):
        super().__init__(SecurityDefinitionType.OPTIONS)
        self.__expiry = 0
        self.__tick_value = 0
        self.__tick_size = 0

    @property
    def expiry(self):
        return self.__expiry

    @expiry.setter
    def expiry(self, expiry):
        self.__expiry = expiry

    @property
    def tick_value(self):
        return self.__tick_value

    @tick_value.setter
    def tick_value(self, tick_value):
        self.__tick_value = tick_value

    @property
    def tick_size(self):
        return self.__tick_size

    @tick_size.setter
    def tick_size(self, tick_size):
        self.__tick_size = tick_size

    def __str__(self):
        return "DoNotImplement"

    def __repr__(self):
        return "DoNotImplement"

class FxSecurityDefinition(SecurityDefinition):
    def __init__(self):
        super().__init__(SecurityDefinitionType.FX)
        self.__pip_scale = 0

    @property
    def pip_scale(self):
        return self.__pip_scale

    @pip_scale.setter
    def pip_scale(self, pip_scale):
        self.__pip_scale = pip_scale

    def __str__(self):
        return "DoNotImplement"

    def __repr__(self):
        return "DoNotImplement"

class InvalidIdException(Exception):
    pass

class MarketDataReceiver:
    def __init__(self, subscriber):
        self.__subscriber = subscriber

    def handleFuturesSecurityDefinition(self, msg):
        if msg.getId() < 0:
            raise InvalidIdException("Negative quantity encountered!")

        dmsg = FuturesSecurityDefinition()
        dmsg.symbol_name = msg.getSymbolName()
        dmsg.id = msg.getId()
        dmsg.tick_value = msg.getTickValue()
        dmsg.tick_size = msg.getTickSize()
        self.__subscriber.addSecurityDefinition(dmsg)

    def handleOptionsSecurityDefinition(self, msg):
        if msg.getId() < 0:
            raise InvalidIdException("Negative quantity encountered!")

        dmsg = OptionsSecurityDefinition()
        dmsg.symbol_name = msg.getSymbolName()
        dmsg.id = msg.getId()
        dmsg.expiry = msg.getExpiry()
        dmsg.tick_value = msg.getTickValue()
        dmsg.tick_size = msg.getTickSize()
        self.__subscriber.addSecurityDefinition(dmsg)

    def handleFxSecurityDefinition(self, msg):
        if msg.getId() < 0:
            raise InvalidIdException("Negative quantity encountered!")

        dmsg = FxSecurityDefinition()
        dmsg.symbol_name = msg.getSymbolName()
        dmsg.id = msg.getId()
        dmsg.pip_scale = msg.getPipScale()
        self.__subscriber.addSecurityDefinition(dmsg)


class MarketDataSubscriber:
    def __init__(self):
        self.__queue = []

    def addSecurityDefinition(self, msg):
        self.__queue.append(msg)

    def printSecurityDefinitions(self):
        for msg in self.__queue:
            if msg.security_definition_type == SecurityDefinitionType.FUTURES:
                print("FuturesSecurityDefinition symbol_name {0} id {1} tick_value {2} tick_size {3}".format(msg.symbol_name, msg.id, msg.tick_value, msg.tick_size))
            elif msg.security_definition_type == SecurityDefinitionType.OPTIONS:
                print("OptionsSecurityDefinition symbol_name {0} id {1} expiry {2} tick_value {3} tick_size {4}".format(msg.symbol_name, msg.id, msg.expiry, msg.tick_value, msg.tick_size))
            else:
                print("FxSecurityDefinition symbol_name {0} id {1} pip_scale {2}".format(msg.symbol_name, msg.id, msg.pip_scale))

def test_feed_futures_constructor():
  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50)
  tick_value = random.randint(50, 100)
  tick_size = random.randint(10, 99) / 100

  msg = FeedFuturesSecurityDefinition(symbol_name, id, tick_value, tick_size)
  assert(msg.getSymbolName() == symbol_name)
  assert(msg.getId() == id)
  assert(msg.getTickValue() == tick_value)
  assert(msg.getTickSize() == tick_size)

def test_feed_options_constructor():
  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50)
  expiry = datetime.datetime(2023, random.randint(0, 12), random.randint(1, 28))
  tick_value = random.randint(50, 100)
  tick_size = random.randint(10, 99) / 100

  msg = FeedOptionsSecurityDefinition(symbol_name, id, expiry, tick_value, tick_size)
  assert(msg.getSymbolName() == symbol_name)
  assert(msg.getId() == id)
  assert(msg.getExpiry() == expiry)
  assert(msg.getTickValue() == tick_value)
  assert(msg.getTickSize() == tick_size)

def test_feed_fx_constructor():
  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50)
  pip_scale = random.randint(10, 99) / 100

  msg = FeedFxSecurityDefinition(symbol_name, id, pip_scale)
  assert(msg.getSymbolName() == symbol_name)
  assert(msg.getId() == id)
  assert(msg.getPipScale() == pip_scale)

def test_futures():
  test_futures_inheritance_check()

  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50)
  tick_value = random.randint(50, 100)
  tick_size = random.randint(10, 99) / 100

  sub = MarketDataSubscriber()
  recv = MarketDataReceiver(sub)
  msg = FeedFuturesSecurityDefinition(symbol_name, id, tick_value, tick_size)
  recv.handleFuturesSecurityDefinition(msg)

  with redirect_stdout(io.StringIO()) as f:
    sub.printSecurityDefinitions()
  s = f.getvalue()

  comp = "FuturesSecurityDefinition symbol_name {0} id {1} tick_value {2} tick_size {3}\n".format(symbol_name, id, tick_value, tick_size)
      
  assert(s == comp)

def test_options():
  test_options_inheritance_check()

  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50)
  expiry = datetime.datetime(2023, random.randint(1, 12), random.randint(1, 28))
  tick_value = random.randint(50, 100)
  tick_size = random.randint(10, 99) / 100

  sub = MarketDataSubscriber()
  recv = MarketDataReceiver(sub)
  msg = FeedOptionsSecurityDefinition(symbol_name, id, expiry, tick_value, tick_size)
  recv.handleOptionsSecurityDefinition(msg)

  with redirect_stdout(io.StringIO()) as f:
    sub.printSecurityDefinitions()
  s = f.getvalue()

  comp = "OptionsSecurityDefinition symbol_name {0} id {1} expiry {2} tick_value {3} tick_size {4}\n".format(symbol_name, id, expiry, tick_value, tick_size)
  assert(s == comp)

def test_fx():
  test_fx_inheritance_check()

  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50)
  pip_scale = random.randint(10, 99) / 100

  sub = MarketDataSubscriber()
  recv = MarketDataReceiver(sub)
  msg = FeedFxSecurityDefinition(symbol_name, id, pip_scale)
  recv.handleFxSecurityDefinition(msg)

  with redirect_stdout(io.StringIO()) as f:
    sub.printSecurityDefinitions()
  s = f.getvalue()

  comp = "FxSecurityDefinition symbol_name {0} id {1} pip_scale {2}\n".format(symbol_name, id, pip_scale)

  assert(s == comp)

def test_multiple():
  inheritance_check()

  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50)
  expiry = datetime.datetime(2023, random.randint(1, 12), random.randint(1, 28))
  tick_value = random.randint(50, 100)
  tick_size = random.randint(10, 99) / 100
  pip_scale = random.randint(10, 99) / 100

  sub = MarketDataSubscriber()
  recv = MarketDataReceiver(sub)

  msg = FeedFuturesSecurityDefinition(symbol_name, id, tick_value, tick_size)
  recv.handleFuturesSecurityDefinition(msg)

  msg = FeedOptionsSecurityDefinition(symbol_name, id, expiry, tick_value, tick_size)
  recv.handleOptionsSecurityDefinition(msg)

  msg = FeedFxSecurityDefinition(symbol_name, id, pip_scale)
  recv.handleFxSecurityDefinition(msg)

  with redirect_stdout(io.StringIO()) as f:
    sub.printSecurityDefinitions()
  s = f.getvalue()

  comp = "FuturesSecurityDefinition symbol_name {0} id {1} tick_value {2} tick_size {3}\n".format(symbol_name, id, tick_value, tick_size)
  comp += "OptionsSecurityDefinition symbol_name {0} id {1} expiry {2} tick_value {3} tick_size {4}\n".format(symbol_name, id, expiry, tick_value, tick_size)
  comp += "FxSecurityDefinition symbol_name {0} id {1} pip_scale {2}\n".format(symbol_name, id, pip_scale)

  assert(s == comp)

def get_writeable_properties(cls):
  return [attr for attr, value in vars(cls).items()
     if isinstance(value, property) and value.fset is not None]

def test_futures_message():
  test_futures_inheritance_check()

  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50)
  tick_value = random.randint(50, 100)
  tick_size = random.randint(10, 99) / 100

  message = FuturesSecurityDefinition()

  message.symbol_name = symbol_name
  message.id = id
  message.tick_value = tick_value
  message.tick_size = tick_size

  assert(message.symbol_name == symbol_name)
  assert(message.id == id)
  assert(message.tick_value == tick_value)
  assert(message.tick_size == tick_size)

def test_options_message():
  test_options_inheritance_check()

  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50)
  expiry = datetime.datetime(2023, random.randint(1, 12), random.randint(1, 28))
  tick_value = random.randint(50, 100)
  tick_size = random.randint(10, 99) / 100

  message = OptionsSecurityDefinition()

  message.symbol_name = symbol_name
  message.id = id
  message.expiry = expiry
  message.tick_value = tick_value
  message.tick_size = tick_size

  assert(message.symbol_name == symbol_name)
  assert(message.id == id)
  assert(message.expiry == expiry)
  assert(message.tick_value == tick_value)
  assert(message.tick_size == tick_size)

def test_fx_message():
  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50)
  pip_scale = random.randint(10, 99) / 100

  message = FxSecurityDefinition()

  message.symbol_name = symbol_name
  message.id = id
  message.pip_scale = pip_scale

  assert(message.symbol_name == symbol_name)
  assert(message.id == id)
  assert(message.pip_scale == pip_scale)

def inheritance_check():
  test_futures_inheritance_check()
  test_options_inheritance_check()
  test_fx_inheritance_check()

def test_futures_inheritance_check():
  message = FuturesSecurityDefinition()

  assert(issubclass(FuturesSecurityDefinition, SecurityDefinition))

  assert(len(get_writeable_properties(FuturesSecurityDefinition)) > 0)

  assert(message.__dict__ == {'_FuturesSecurityDefinition__tick_value': 0, '_FuturesSecurityDefinition__tick_size': 0, '_SecurityDefinition__symbol_name': "", '_SecurityDefinition__id': 0, '_SecurityDefinition__security_definition_type': SecurityDefinitionType.FUTURES})

  assert(message.__repr__() == "DoNotImplement")
  assert(message.__str__() == "DoNotImplement")

def test_options_inheritance_check():
  message = OptionsSecurityDefinition()

  assert(issubclass(OptionsSecurityDefinition, SecurityDefinition))

  assert(len(get_writeable_properties(OptionsSecurityDefinition)) > 0)

  assert(message.__dict__ == {'_OptionsSecurityDefinition__tick_value': 0, '_OptionsSecurityDefinition__tick_size': 0, '_OptionsSecurityDefinition__expiry': 0, '_SecurityDefinition__symbol_name': "", '_SecurityDefinition__id': 0, '_SecurityDefinition__security_definition_type': SecurityDefinitionType.OPTIONS})

  assert(message.__repr__() == "DoNotImplement")
  assert(message.__str__() == "DoNotImplement")

def test_fx_inheritance_check():
  message = FxSecurityDefinition()

  assert(issubclass(FxSecurityDefinition, SecurityDefinition))

  assert(len(get_writeable_properties(FxSecurityDefinition)) > 0)

  assert(message.__dict__ == {'_FxSecurityDefinition__pip_scale': 0, '_SecurityDefinition__symbol_name': "", '_SecurityDefinition__id': 0, '_SecurityDefinition__security_definition_type': SecurityDefinitionType.FX})

  assert(message.__repr__() == "DoNotImplement")
  assert(message.__str__() == "DoNotImplement")

def test_negative_id_futures():
  test_futures_inheritance_check()

  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50) * -1
  tick_value = random.randint(50, 100)
  tick_size = random.randint(10, 99) / 100

  sub = MarketDataSubscriber()
  recv = MarketDataReceiver(sub)
  msg = FeedFuturesSecurityDefinition(symbol_name, id, tick_value, tick_size)

  try:
    recv.handleFuturesSecurityDefinition(msg)
  except InvalidIdException:
    assert(True)
  else:
    assert(False)

def test_negative_id_options():
  test_options_inheritance_check()

  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50) * -1
  expiry = datetime.datetime(2023, random.randint(1, 12), random.randint(1, 28))
  tick_value = random.randint(50, 100)
  tick_size = random.randint(10, 99) / 100

  sub = MarketDataSubscriber()
  recv = MarketDataReceiver(sub)
  msg = FeedOptionsSecurityDefinition(symbol_name, id, expiry, tick_value, tick_size)

  try:
    recv.handleOptionsSecurityDefinition(msg)
  except InvalidIdException:
    assert(True)
  else:
    assert(False)

def test_negative_id_fx():
  test_options_inheritance_check()

  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50) * -1
  pip_scale = random.randint(10, 99) / 100

  sub = MarketDataSubscriber()
  recv = MarketDataReceiver(sub)
  msg = FeedFxSecurityDefinition(symbol_name, id, pip_scale)

  try:
    recv.handleFxSecurityDefinition(msg)
  except InvalidIdException:
    assert(True)
  else:
    assert(False)

@negative_value_checker
def test_decorator_1(var1, var2, var3, var4):
  assert(False)

@negative_value_checker
def test_decorator_2(var1, var2, var3, var4, var5):
  assert(False)

@negative_value_checker
def test_decorator_3(var1, var2, var3):
  assert(False)

def test_decorator():
  symbol_name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
  id = random.randint(1, 50)
  expiry = datetime.datetime(2023, random.randint(1, 12), random.randint(1, 28))
  tick_value = random.randint(50, 100)
  tick_size = random.randint(10, 99) / -100
  pip_scale = random.randint(10, 99) / -100

  sub = MarketDataSubscriber()
  recv = MarketDataReceiver(sub)

  try:
    test = FeedFuturesSecurityDefinition(symbol_name, id, tick_value, tick_size)
    assert(False)
  except:
    assert(True)

  try:
    test = FeedOptionsSecurityDefinition(symbol_name, id, expiry, tick_value, tick_size)
    assert(False)
  except:
    assert(True)

  try:
    test = FeedFxSecurityDefinition(symbol_name, id, pip_scale)
    assert(False)
  except:
    assert(True)

  try:
    test_decorator_1(symbol_name, id, tick_value, tick_size)
    assert(False)
  except:
    assert(True)

  try:
    test_decorator_2(symbol_name, id, expiry, tick_value, tick_size)
    assert(False)
  except:
    assert(True)

  try:
    test_decorator_3(symbol_name, id, pip_scale)
    assert(False)
  except:
    assert(True)

if __name__ == "__main__":
    test_futures_inheritance_check()
    test_options_inheritance_check()
    test_fx_inheritance_check()

    test_futures_message()
    test_options_message()
    test_fx_message()

    test_feed_futures_constructor()
    test_feed_options_constructor()
    test_feed_fx_constructor()

    test_multiple()

    test_futures()
    test_options()
    test_fx()

    test_negative_id_futures()
    test_negative_id_options()
    test_negative_id_fx()

    test_decorator()


In [ ]:
######### my version (passed 11/14) #########

class SecurityDefinition:
  # Implement class functions
    def __init__(self,security_definition_type):
        
        self.__security_definition_type = security_definition_type
        self.__id = 0
        self.__symbol_name = ""
        
    @property
    def security_definition_type(self):
        return self.__security_definition_type 
    
    @security_definition_type.setter
    def security_definition_type(self, security_definition_type):
        self.__security_definition_type = security_definition_type 
        
    @property
    def id(self):
        return self.__id
    
    @id.setter
    def id(self, id):
        self.__id  = id
        
    @property
    def symbol_name(self):
        return self.__symbol_name
    
    @symbol_name.setter
    def symbol_name(self, symbol_name):
        self.__symbol_name  = symbol_name
        
        
class FuturesSecurityDefinition(SecurityDefinition):
  # Implement class functions
    def __init__(self):

        super().__init__(SecurityDefinitionType.FUTURES)
        self.__tick_value = 0
        self.__tick_size = 0
    
    @property
    def tick_value(self):
        return self.__tick_value

    @tick_value.setter
    def tick_value(self, tick_value):
        self.__tick_value = tick_value
        
    @property
    def tick_size(self):
        return self.__tick_size

    @tick_size.setter
    def tick_size(self, tick_size):
        self.__tick_size = tick_size

    def __str__(self):
        return "DoNotImplement"

    def __repr__(self):
        return "DoNotImplement"

        
class OptionsSecurityDefinition(SecurityDefinition):
  # Implement class functions
    def __init__(self):
        
        super().__init__(SecurityDefinitionType.OPTIONS)
        self.__expiry = 0
        self.__tick_value = 0
        self.__tick_size = 0
        
    @property
    def expiry(self):
        return self.__expiry

    @expiry.setter
    def expiry(self, expiry):
        self.__expiry = expiry
        
    @property
    def tick_value(self):
        return self.__tick_value

    @tick_value.setter
    def tick_value(self, tick_value):
        self.__tick_value = tick_value
        
    @property
    def tick_size(self):
        return self.__tick_size

    @tick_size.setter
    def tick_size(self, tick_size):
        self.__tick_size = tick_size

        
    def __str__(self):
        return "DoNotImplement"

    def __repr__(self):
        return "DoNotImplement"

        
class FxSecurityDefinition(SecurityDefinition):
  # Implement class functions
    def __init__(self):
        
        super().__init__(SecurityDefinitionType.FX)
        self.__pip_scale = 0
        
    @property
    def pip_scale(self):
        return self.__pip_scale

    @pip_scale.setter
    def pip_scale(self, pip_scale):
        self.__pip_scale = pip_scale

    def __str__(self):
        return "DoNotImplement"

    def __repr__(self):
        return "DoNotImplement"

        
#Implement InvalidIdException

class MarketDataReceiver:
    # Implement class functions
    def __init__(self,sub):
        self.__sub = sub

    def handleFuturesSecurityDefinition(self, msg):
        
        f = FuturesSecurityDefinition()
        f.symbol_name = msg.getSymbolName()
        f.id = msg.getId()
        f.tick_value = msg.getTickValue()
        f.tick_size = msg.getTickSize()
        self.__sub.addSecurityDefinition(f)
        
    def handleOptionsSecurityDefinition(self, msg):
        
        f = OptionsSecurityDefinition()
        f.symbol_name = msg.getSymbolName()
        f.id = msg.getId()
        f.tick_value = msg.getTickValue()
        f.tick_size = msg.getTickSize()
        f.expiry = msg.getExpiry()
        self.__sub.addSecurityDefinition(f)

    def handleFxSecurityDefinition(self, msg):
        
        f = FxSecurityDefinition
        f.symbol_name = msg.getSymbolName()
        f.id = msg.getId()
        f.pip_scale = msg.getPipScale()
        self.__sub.addSecurityDefinition(f)


class MarketDataSubscriber:
    # Implement class functions
    def __init__(self):
        self.__queue = []

    def addSecurityDefinition(self, msg):
        self.__queue.append(msg)

    def printSecurityDefinitions(self):
        
        # "FuturesSecurityDefinition symbol_name {0} id {1} tick_value {2} tick_size {3}\n".format(symbol_name, id, tick_value, tick_size)
        
        for msg in self.__queue:
            
            if msg.security_definition_type == SecurityDefinitionType.FUTURES:
                
                print("FuturesSecurityDefinition symbol_name "+msg.symbol_name+" id "+str(msg.id)+" tick_value " +str(msg.tick_value)+" tick_size "+str(msg.tick_size))
                # print("FuturesSecurityDefinition symbol_name {0} id {1} tick_value {2} tick_size {3}".format(msg.symbol_name, msg.id, msg.tick_value, msg.tick_size))
            
            elif msg.security_definition_type == SecurityDefinitionType.OPTIONS:
                print("OptionsSecurityDefinition symbol_name "+msg.symbol_name+" id "+str(msg.id)+" expiry "+str(msg.expiry)+" tick_value " +str(msg.tick_value)+" tick_size "+str(msg.tick_size))
                # print("OptionsSecurityDefinition symbol_name {0} id {1} expiry {2} tick_value {3} tick_size {4}".format(msg.symbol_name, msg.id, msg.expiry, msg.tick_value, msg.tick_size))
            
            else:
                print("FxSecurityDefinition symbol_name {0} id {1} pip_scale {2}".format(msg.symbol_name, msg.id, msg.pip_scale))

## Q2

In [ ]:
#question 2
  def getTime(s):
    # Write your code here
    ans = 0
    curr = ord('A')
    for i in range(len(s)):
        k = ord(s[i])
        a = abs(curr - k)
        b = 26 - abs(curr - k)
        ans += min(a, b)

        curr = ord(s[i])
    return ans